stack模型集成

In [ ]:
import numpy as np

我们需要输入三个模形的OOF验证集，以及预测集，还有训练集的真实target，最后就可以得到结果

In [ ]:
def stack_model(oof1,oof2,oof3,sub1,sub2,sub3,train_target):
    #oof代表每个模型的验证集，sub代表每个模型在预测集上的结果
    #train_stack是指按列进行拼接，代表三个模型在数据集上的验证结果
    train_stack=np.hstack([oof1,oof2,oof3])
    #test_stack是指按列进行拼接，代表三个模型在测试集上的结果
    test_stack=np.hstack([sub1,sub2,sub3])
    #创建一个和测试集的行数相同的全零数组
    prediction=np.zeros(test_stack.shape[0])
    #创建一个和训练集的行数相同的全零数组
    OOF=np.zeros(train_stack.shape[0])
    #多轮交叉验证stack
    from sklearn.model_selection import RepeatedKFold
    folds=RepeatedKFold(n_splits=5,n_repeats=2,random_state=2020)

    for fold_,(trn_idx,val_idx) in enumerate(folds.split(train_stack,train_target)):
        print("foln n{}".format(fold_+1))
        trn_data,trn_y=train_stack[trn_idx],train_target[trn_idx]
        val_data,val_y=train_stack[val_idx],train_target[val_idx]
        print("-"*10+'Stacking'+str(fold_+1)+'-'*10)
        #使用贝叶斯回归作为stack模型
        from sklearn.linear_model import BayesianRidge
        clf=BayesianRidge()
        clf.fit(trn_data,trn_y)
        #在训练集上进行训练
        OOF[val_idx]=clf.predict(val_data)/(5*2)
        #在验证机上做预测
        prediction+=clf.predict(test_stack)/(5*2)
        #对测试集数据进行训练，每一轮结果占比额外的1/10
    #返回测试集和验证集
    return OOF,prediction
    